In [ ]:
%matplotlib qt

from pathlib import Path
from time import time, sleep
import re 
from math import isnan

import pandas as pd
from pandas.api.types import is_list_like

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import numpy as np

from pdfminer.layout import LAParams, LTTextBox, LTRect, LTTextLine, LTCurve
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator

from tqdm import tqdm

from pdf2image import convert_from_path

from notify_run import Notify

import json 

In [ ]:
notify = Notify()
notify.register()

# Funções

In [ ]:
def extract_elements(fname):
    with open(fname, 'rb') as fp:
        rsrcmgr = PDFResourceManager()
        laparams = LAParams()
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        pages = PDFPage.get_pages(fp)

        x0, y0, x1, y1 = [], [], [], []
        txt = []
        pgn = []
        element_type = []

        for i, page in enumerate(tqdm(pages)):
 #           print(f'Processing page {i}...')
            interpreter.process_page(page)
            layout = device.get_result()
            for lobj in layout:
                if isinstance(lobj, LTTextBox):
                    for text_line in lobj:
                        if isinstance(text_line, LTTextLine):
                            text = text_line.get_text().strip()
                            if text:
                                x0.append(text_line.bbox[0])
                                y0.append(text_line.bbox[1])
                                x1.append(text_line.bbox[2])
                                y1.append(text_line.bbox[3])
                                txt.append(text)
                                pgn.append(i)
                                element_type.append("text_line")
                elif isinstance(lobj, LTRect):
                    x, y = lobj.bbox[0], lobj.bbox[3]
                    x0.append(lobj.bbox[0])
                    y0.append(lobj.bbox[1])
                    x1.append(lobj.bbox[2])
                    y1.append(lobj.bbox[3])
                    txt.append("")
                    pgn.append(i)
                    element_type.append("rectangle")
                elif isinstance(lobj, LTCurve):
                    x, y = lobj.bbox[0], lobj.bbox[3]
                    x0.append(lobj.bbox[0])
                    y0.append(lobj.bbox[1])
                    x1.append(lobj.bbox[2])
                    y1.append(lobj.bbox[3])
                    txt.append("")
                    pgn.append(i)
                    element_type.append("curve")
                    
    return pd.DataFrame({"element_type": element_type, "x0":x0, "y0":y0, "x1": x1, "y1": y1, "page": pgn, "text": txt,})

In [ ]:
def get_element_by_text(elements, text, page=None):
    if page is not None:
        elements = elements[elements.page == page]
    elements = elements[elements.element_type == "text_line"]
    if not is_list_like(text):
        text = [text]
    return elements[elements.text.str.strip().str.replace(r"\s+", " ").isin(text)].iloc[0] 

def get_right(elements, field_element, page=None):
    if page is not None:
        elements = elements[elements.page == page]    
    y_ = (field_element.y0 + field_element.y1)/2
    return elements[(elements.y0 < y_) & (elements.y1 > y_) & (elements.x0 > field_element.x1)].sort_values("x0")

def get_above(elements, reference_element, page=None):
    if page is not None:
        elements = elements[elements.page == page]    
    x_ = reference_element.x0
    return elements[(elements.x0 < x_) & (elements.x1 > x_) & (elements.y0 > reference_element.y1)].sort_values("y0")

def get_left(elements, check_element, page=None):
    if page is not None:
        elements = elements[elements.page == page]
    y_ = (check_element.y0 + check_element.y1)/2
    return elements[(elements.y0 < y_) & (elements.y1 > y_) & (elements.x0 < check_element.x0)].sort_values("x0", ascending=False)    
    

def clip(elements, left=None, right=None, top=None, bottom=None, page=None):
    if page is not None:
        elements = elements[elements.page == page]    
    if left is not None:
        elements = elements[elements.x0 > left]
    if right is not None:
        elements = elements[elements.x0 < right]        
    if top is not None:
        elements = elements[elements.y1 < top]    
    if bottom is not None:
        elements = elements[elements.y1 > bottom] 
    return elements

def get_check_label(elements, check_element, page=None):
    if page is not None:
        elements = elements[elements.page == page]
    y_ = (check_element.y0 + check_element.y1)/2
    return elements[(elements.y0 < y_) & (elements.y1 > y_) & (elements.x0 > check_element.x1)].sort_values("x0").text.iloc[0]    
    
   
        
def plot_page(fname, page=0, dpi=200, plot_txt=None, ax=None, alpha=0.5, color="blue", grid=True):
    if ax is None:
        fig = plt.figure()
        ax = fig.gca()
    img = convert_from_path(fname, dpi=dpi, first_page=page+1, last_page=page+1)[0]
    h, w = img.height, img.width
    extent = (0, w*72/dpi, 0, h*72/dpi)
    
    ax.imshow(img, extent=extent)
    ax.grid(grid)
    
    if plot_txt is not None:
        for i, r in plot_txt[plot_txt.page == page].iterrows():
            ax.add_patch(Rectangle((r.x0, r.y0), r.x1 - r.x0, r.y1 - r.y0, alpha=alpha, color=color, picker=True))
            
    return ax

In [ ]:
class Alarm:
    def __init__(self, message):
        self.message = message
    def __enter__(self):
        self.start_time = time()
    def __exit__(self, type_, value, traceback):
        dt = time()-self.start_time
        if value is None:
            notify.send(f"{self.message} Terminado em {dt/60} minutos")
        else:
            notify.send(f"Erro {value} processando {self.message}")

# Processamento

In [ ]:
acs = "Municípios.pdf" 

elements = extract_elements(acs)

elements.to_parquet("municipios.parquet", compression=None)

In [ ]:
areas_contaminadas = []

elements = elements[elements.element_type != "rectangle"]

with Alarm("ACs Cetesb 2019"):
    for page in tqdm(range(elements.page.max()+1)):
        dados = {
            "empreendimento":None,
            "endereco":None,
            "municipio":None,
            "atividade":None,
            "coordenadas":None,
            "classificacao":None,
            "reutilizacao":False,
            "etapas":None,
            "fontes":None,
            "meios_impactados":None,
            "contaminantes":None,
            "fase_livre":False,
            "pops":False,
            "medidas_emergenciais":None,
            "medidas_controle_institucional":None,
            "medidas_remediacao":None,
            "medidas_controle_engenharia":None   
        }
        
        page_elements = elements[elements.page == page]
        page_text = page_elements[page_elements.element_type == "text_line"]
        page_check = page_elements[page_elements.element_type == "curve"]
        page_check = page_check[abs((page_check.x0 - page_check.x1)/(page_check.y0 - page_check.y1) - 1) < 0.5]
        
        atividade = get_element_by_text(page_text, "Atividade")
        dados_empreendimento = get_above(page_text, atividade).text.to_list()
        dados["empreendimento"] = dados_empreendimento[1]
        dados["endereco"] = dados_empreendimento[0]
        if len(dados_empreendimento) == 3:
            dados["municipio"] = dados_empreendimento[2]
        
        dados["atividade"] = get_check_label(page_text, get_right(page_check, atividade).iloc[0])
        dados["coordenadas"] = get_right(page_text, get_element_by_text(page_text, "Coordenadas (m):")).text.str.cat(sep = " ")
        
        linha_classificacao = get_right(page_elements, get_element_by_text(page_text, "Classificação"))
        dados["classificacao"] = linha_classificacao.text.iloc[0]
        dados["reutilizacao"] = "curve" in linha_classificacao.element_type
        
        etapas_titulo = get_element_by_text(page_text, "Etapas do gerenciamento") 
        fonte_titulo = get_element_by_text(page_text, "Fonte de contaminação")

        etapas = clip(page_check, top=etapas_titulo.y0, bottom=fonte_titulo.y1)
        dados["etapas"] = [
            get_check_label(page_text, check)
            for check in etapas.itertuples(index=False)
        ]
        
        meios_titulo = get_element_by_text(page_text, "Meios impactados")
        
        meios = clip(page_check, top=fonte_titulo.y0, bottom=meios_titulo.y1)
        dados["fontes"] = [
            get_check_label(page_text, check)
            for check in meios.itertuples(index=False)
        ]
        
        institucional_titulo =  get_element_by_text(page_text, "Medidas de controle institucional")
        contaminantes_titulo = get_element_by_text(page_text, "Contaminantes")
        
        contaminantes = clip(page_check, top=contaminantes_titulo.y0, bottom=institucional_titulo.y1, left=contaminantes_titulo.x0)
        dados["contaminantes"] = [
            get_check_label(page_text, check)
            for check in contaminantes.itertuples(index=False)
        ]
        
        emergencial_titulo =  get_element_by_text(page_text, "Medidas emergenciais")
        remediacao_titulo = get_element_by_text(page_text, "Medidas de remediação")
        trabalhadores_obras_titulo = get_element_by_text(page_text, "trabalhadores de obras")
        
        emergencial = clip(page_check, top=emergencial_titulo.y0, bottom=remediacao_titulo.y1, right=trabalhadores_obras_titulo.x0)
        dados["medidas_emergenciais"] = [
            get_check_label(page_text, check)
            for check in emergencial.itertuples(index=False)
        ]
        
        engenharia_titulo = get_element_by_text(page_text, "Medidas de controle de engenharia")
        
        remediacao = clip(page_check, top=remediacao_titulo.y0, bottom=engenharia_titulo.y1)
        dados["medidas_remediacao"] = [
            get_check_label(page_text, check)
            for check in remediacao.itertuples(index=False)
        ]  
        
        engenharia = clip(page_check, top=engenharia_titulo.y0)
        dados["medidas_controle_engenharia"] = [
            get_check_label(page_text, check)
            for check in engenharia.itertuples(index=False)
        ]        
        
        faselivre_titulo = get_element_by_text(page_text, "existência de fase livre")
        
        impactados = clip(page_check, top=meios_titulo.y0, bottom=faselivre_titulo.y1, right=trabalhadores_obras_titulo.x0)
        dados["meios_impactados"] = [
            (get_above(page_text, check).iloc[0].text, get_left(page_text, check).iloc[0].text)
            for check in impactados.itertuples(index=False)
        ]  

        institucional = clip(page_check, top=institucional_titulo.y0, bottom=trabalhadores_obras_titulo.y1, left=trabalhadores_obras_titulo.x0)
        dados["medidas_controle_institucional"] = [
            (get_above(page_text, check).iloc[0].text, get_left(page_text, check).iloc[0].text)
            for check in institucional.itertuples(index=False)
        ]          
              
        dados["fase_livre"] = not get_left(page_check, faselivre_titulo).empty
        dados["pops"] = not get_left(page_check, get_element_by_text(page_text, "existência de POPs")).empty        
        
        areas_contaminadas.append(dados)

In [ ]:
cetesb = pd.DataFrame(areas_contaminadas)
cetesb["municipio"] = cetesb["municipio"].fillna(method="ffill")

In [ ]:
cetesb.to_excel("cetesb2019.xlsx")

# Áreas Contaminadas extraídas

In [ ]:
acs.columns

In [ ]:
acs = pd.read_excel("G:\Meu Drive\8th Semestre\TF\cetesb2019.xlsx", index_col=0)

In [ ]:
def filtrar_duplas(valor):
    def filtro(lista):
        return [b for a, b in lista if a == valor]
    return filtro

In [ ]:
meios_impactados = acs.meios_impactados.str.replace("'", "\"", regex=False).str.replace("(", "[", regex=False).str.replace(")", "]", regex=False).map(json.loads)

dentro = meios_impactados.map(filtrar_duplas("Dentro")).map(", ".join)
acs["meio_interno_impactado"] = dentro

fora = meios_impactados.map(filtrar_duplas("Fora")).map(", ".join)
acs["meio_externo_impactado"] = fora

In [ ]:
controle_institucional = acs.medidas_controle_institucional.str.replace("'", "\"", regex=False).str.replace("(", "[", regex=False).str.replace(")", "]", regex=False).map(json.loads)

plano = controle_institucional.map(filtrar_duplas("ou no plano de intervenção")).map(", ".join)
acs["medida_adm_proposta"] = plano

responsavel  = controle_institucional.map(filtrar_duplas("responsável")).map(", ".join)
acs["medida_adm_comunidada"] = responsavel

implantada  = controle_institucional.map(filtrar_duplas("implantada")).map(", ".join)
acs["medida_adm_implantada"] = implantada

In [ ]:
etapas = acs.etapas.str.replace("'", "\"", regex=False).str.replace("(", "[", regex=False).str.replace(")", "]", regex=False).map(json.loads)
gerenciamento = etapas.map(", ".join)
acs["etapas"] = gerenciamento

In [ ]:
fonte = acs.fontes.str.replace("'", "\"", regex=False).str.replace("(", "[", regex=False).str.replace(")", "]", regex=False).map(json.loads)
fontes = fonte.map(", ".join)
acs["fontes"] = fontes

In [ ]:
rem = acs.medidas_remediacao.str.replace("'", "\"", regex=False).str.replace("(", "[", regex=False).str.replace(")", "]", regex=False).map(json.loads)
remediacao = rem.map(", ".join)
acs["medida_remediacao"] = remediacao

In [ ]:
eng = acs.medidas_controle_engenharia.str.replace("'", "\"", regex=False).str.replace("(", "[", regex=False).str.replace(")", "]", regex=False).map(json.loads)
engenharia = eng.map(", ".join)
acs["medida_engenharia"] = engenharia

In [ ]:
cont = acs.contaminantes.str.replace("'", "\"", regex=False).str.replace("(", "[", regex=False).str.replace(")", "]", regex=False).map(json.loads)
contaminante = cont.map(", ".join)
acs["contaminantes"] = contaminante

In [ ]:
em = acs.medidas_emergenciais.str.replace("'", "\"", regex=False).str.replace("(", "[", regex=False).str.replace(")", "]", regex=False).map(json.loads)
emergencia = em.map(", ".join)
acs["medida_emergencial"] = emergencia

In [ ]:
company = (acs.empreendimento
           .str.strip()
           .str.replace(r"\s+", r" ", regex=True)
)

In [ ]:
adress = (acs.endereco
           .str.strip()
           .str.replace(r"\s+", r" ", regex=True)
)

In [ ]:
district = (acs.municipio
           .str.strip()
           .str.replace(r"\s+", r" ", regex=True)
) 

In [ ]:
coord = acs.coordenadas.str.extract(r"(?P<crs>fuso [0-9]+ DATUM [0-9A-Za-zó ]+) UTM_E (?P<leste>[0-9.]+),00 UTM_N (?P<norte>[0-9.]+),00")

In [ ]:
acs["crs"] = coord.crs.str.replace(r"^fuso ([0-9]+) DATUM ([0-9A-Za-zó ]+)", r"UTM \2 \1S", regex=True)
acs["leste"] = pd.to_numeric(coord.leste.str.replace(r"\.", r"", regex=True), downcast='float')
acs["norte"] = pd.to_numeric(coord.norte.str.replace(r"\.", r"", regex=True), downcast='float')
acs["empreendimento"] = company
acs["endereco"] = adress
acs["municipio"] = district

In [ ]:
acs = acs.drop(columns=["meios_impactados", "medidas_emergenciais", "medidas_controle_institucional", "medidas_remediacao", "medidas_controle_engenharia", "coordenadas"])

In [ ]:
columnsTitles = ['empreendimento', 'endereco', 'municipio', 'atividade', 'crs', 'leste', 'norte', 'classificacao','reutilizacao',
                 'etapas', 'fontes', 'meio_interno_impactado', 'meio_externo_impactado', 'contaminantes','fase_livre', 'pops',
                 'medida_emergencial', 'medida_adm_proposta','medida_adm_comunidada', 'medida_adm_implantada', 'medida_remediacao','medida_engenharia']

acs_corrigido = acs.reindex(columns=columnsTitles)

In [ ]:
acs_corrigido.to_excel("cetesb2019_corrigido.xlsx")

In [ ]:
lista_dentro = []
lista_fora = []
for tupla in acs.meios_impactados:
    if tupla[3:][0] == 'Dentro':
        lista_dentro.append(tupla)
    elif tupla[3:][0] == 'Fora':
        lista_fora.append(tupla)
print(lista_dentro)

# GeoDataFrame

In [ ]:
%matplotlib inline

import geopandas as gpd
import pandas as pd

from shapely.geometry import Polygon, Point, LineString, MultiPolygon

import os

import fiona


In [ ]:
acs_corrigido = pd.read_excel("cetesb2019_corrigido.xlsx", index_col=0)

In [ ]:
acs_corrigido["crs_code"] = acs_corrigido.crs
        
     

In [ ]:
geo_dados = gpd.GeoDataFrame(acs_corrigido, crs=acs_corrigido.crs, geometry=acs_corrigido) 

In [ ]:
acs["crs_code"] = crs_code

In [ ]:
acs_corrigido["crs_code"] = acs_corrigido.crs_code.replace(
    {"UTM WGS84 23S":"EPSG:32723", "UTM WGS84 22S":"EPSG:32722", "UTM SAD69 23S":"EPSG:29193", "UTM SAD69 22S":"EPSG:29192",
     "UTM Córrego Alegre 22S":"EPSG:22522", "UTM Córrego Alegre 23S":"EPSG:22523", "UTM SIRGAS2000 23S":"EPSG:31983", "UTM SIRGAS2000 22S":"EPSG:31982"})

In [ ]:
columnsTitles = ['empreendimento', 'endereco', 'municipio', 'atividade', 'crs', 'crs_code', 'leste', 'norte', 'classificacao','reutilizacao',
                 'etapas', 'fontes', 'meio_interno_impactado', 'meio_externo_impactado', 'contaminantes','fase_livre', 'pops',
                 'medida_emergencial', 'medida_adm_proposta','medida_adm_comunidada', 'medida_adm_implantada', 'medida_remediacao','medida_engenharia']

acs_corrigido = acs_corrigido.reindex(columns=columnsTitles)

In [ ]:
lista = []

for crs, geodata in acs_corrigido.groupby("crs_code"):
    gdf = gpd.GeoDataFrame(geodata, geometry=gpd.points_from_xy(geodata.leste, geodata.norte), crs=crs)
    sirgas = gdf.to_crs(epsg=4989)
    lista.append(sirgas)
geodados = pd.concat(lista)

In [ ]:
geodados.to_file("acs2019_cetesb.gpkg", driver="GPKG")

In [ ]:
geodados.to_file("acs2019_cetesb.shp")

In [ ]:
fiona.supported_drivers['KML'] = 'rw'

geodados.to_file('acs2019_cetesb.kml', driver='KML')